In [69]:
import nbimporter
import pandas as pd
import numpy as np
import warnings
import util_data_cleaner
import fetch_data_from_yfinance
warnings.filterwarnings('ignore')
from importlib import reload
reload(util_data_cleaner)
reload(fetch_data_from_yfinance)
from datetime import date
import yfinance as yf

In [31]:
ipo_listing_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\config\IPO-PastIssue-23-Aug-2024.csv')
ipo_listing_data = util_data_cleaner.clean_ipo_symbol_data(ipo_listing_data)
ipo_listing_data = ipo_listing_data.loc[ipo_listing_data['DATE OF LISTING']>='2020-01-01',]

In [122]:
b = []
ipo_list = ipo_listing_data['Symbol'].unique().tolist()
for stock in ipo_list:
    start_date = pd.to_datetime(ipo_listing_data.loc[ipo_listing_data['Symbol']==stock,'DATE OF LISTING'].values[0])
    end_date = date.today()
    interval = '1d'
    a = fetch_data_from_yfinance.data_puller(stock, start_date, end_date, interval)
    b.append(a)
final_data = pd.concat(b)
final_data.reset_index(inplace=True)


1 Failed download:
['BALCO.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['SUNLITE.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['POSITRON.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['AESTHETIK.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['PPSL.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['DHARIWAL.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['UTSSAV.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['ASHALOG.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['SSEGL.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['BULKCORP.NS']: Exception('%ticker%: No timezone found, symbol may be deli

In [123]:
final_data.sort_values(by=['Stock Name', 'Date'], inplace=True)
final_data['Ipo_price'] = final_data.groupby('Stock Name')['Adj Close'].transform('first')
final_data['lower_bound'] = final_data['Ipo_price'] * 0.9
final_data['upper_bound'] = final_data['Ipo_price'] * 1.1
latest_close = final_data.groupby('Stock Name').tail(1)
latest_close['to_filter'] = 0
latest_close.loc[((latest_close['Adj Close'] >= latest_close['lower_bound']) & (latest_close['Adj Close'] <= latest_close['upper_bound'])),'to_filter'] = 1

final_data['away_from_ipo_price'] = final_data['Adj Close']/final_data['Ipo_price'] - 1
final_data['away_from_ipo_price_filter'] = final_data['away_from_ipo_price'].apply(lambda x: 1 if(x>0) else 0)
inter = final_data.groupby(['Stock Name'])['away_from_ipo_price_filter'].sum().reset_index()
latest_close = latest_close.merge(inter, how='left', on='Stock Name')

In [145]:
latest_close.to_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\strategy results\ipo_bo_23_aug_2024.csv', index=False)

In [146]:
yf.download('zomato.NS')

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-23,116.000000,138.899994,115.000000,126.000000,126.000000,694895290
2021-07-26,126.349998,143.750000,125.300003,140.649994,140.649994,249723854
2021-07-27,141.699997,147.800003,127.750000,132.899994,132.899994,240341900
2021-07-28,131.000000,135.000000,123.550003,131.199997,131.199997,159793731
2021-07-29,134.949997,144.000000,132.199997,141.550003,141.550003,117973089
...,...,...,...,...,...,...
2024-08-19,278.000000,280.899994,261.000000,262.170013,262.170013,80435895
2024-08-20,262.000000,264.450012,258.250000,263.119995,263.119995,132396556
2024-08-21,263.119995,264.450012,259.000000,260.029999,260.029999,38605742
